In [2]:
import torch
import numpy as np


/root/anaconda3/envs/foldingdiff/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
method = ''
result = torch.load(f"/gaozhangyang/experiments/OpenCPD/results/MPNN/AlphaDesign/results_casp15.pt")
temperature = 0.1 # temperature取0(使用argmax采样), 0.1, 0.5, 1.0

tokenizer = result['tokenizer']
alphabet='ACDEFGHIKLMNPQRSTVWY'
mask_token = []
for token, idx in tokenizer.get_vocab().items():
    if token not in alphabet:
        mask_token.append(idx)

raw_probs = result['pred_probs'][0]
raw_probs[:, mask_token] = -10000
if temperature==0:
    probs = raw_probs
else:
    probs = torch.softmax(raw_probs/temperature, dim=-1)



In [38]:
all_rmsd = []
all_div = []
for iter in range(10):
    if temperature==0:
        pred_seq_id = torch.argmax(probs, 1).squeeze(-1)
    else:
        pred_seq_id = torch.multinomial(probs, 1).squeeze(-1)
    true_seq = tokenizer.decode(result['true_seq'][0]).replace(" ", "")
    pred_seq = tokenizer.decode(pred_seq_id).replace(" ", "")
    rmsd = Cal_RMSD(true_seq, pred_seq)
    diversity = diversity = np.array([ true_res!=pred_res for true_res, pred_res in zip(true_seq, pred_seq)]).mean()
    all_rmsd.append(rmsd)
    all_div.append(diversity)

In [50]:
# 横轴为diversity纵轴为RMSD, 每个temperature下采样10个点，绘制散点图。不同temperature的点用不同颜色标明。
